<a href="https://colab.research.google.com/github/Alexandre77777/neural_networks/blob/main/11.%20%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0%20%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%20%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0.%20%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D0%B5%20%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D1%8B%D0%B5%20%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%20(LLM)/%D0%A2%D0%B5%D0%BC%D0%B0_%E2%84%9612_%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D1%87%D0%B0%D1%82_%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_RAG_%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B7%D0%B5_%D0%BB%D0%BE%D0%BA%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Тема №12. Создание интеллектуальной системы на основе больших языковых моделей (LLM) с применением технологии Retrieval Augmented Generation (RAG)**

### **Введение**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот блокнот посвящен практической реализации системы типа "вопрос-ответ" (Question Answering, QA), основанной на взаимодействии Большой Языковой Модели (LLM) с внешней базой знаний, представленной PDF-документом.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Мы рассмотрим архитектурный подход **Retrieval-Augmented Generation (RAG)**, или **Генерация с Дополненной Выборкой**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот метод позволяет динамически обогащать контекст, подаваемый на вход LLM, релевантной информацией перед тем, как модель сгенерирует ответ.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Такой подход является основным для создания систем, способных оперировать актуальными или специфическими данными, которые не входили в исходный набор данных для обучения LLM.

### **1. Установка и импорт необходимых пакетов**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Современные системы искусственного интеллекта, особенно в области обработки естественного языка (Natural Language Processing, NLP), строятся с использованием специализированных программных библиотек и фреймворков.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Эти инструменты предоставляют удобные средства для взаимодействия с моделями, обработки данных и построения сложных конвейеров обработки (pipelines). В нашем проекте мы будем использовать:
*   **`LangChain`**: Фреймворк для оркестрации компонентов приложений на базе LLM, упрощающий управление промптами, интеграцию с источниками данных и моделями.
*   **`Ollama`**: Среда для локального развертывания и запуска LLM, позволяющая работать с моделями без обращения к облачным сервисам.
*   **`ChromaDB`**: Векторная база данных, оптимизированная для задач поиска по семантической близости – ключевой компонент для RAG.
*   **`Gradio`**: Библиотека для быстрого создания интерактивных веб-интерфейсов, удобная для демонстрации работы модели.
*   **`PyMuPDF`**: Библиотека для извлечения текста из PDF-документов.

Для начала установим и импортируем эти компоненты.

In [1]:
%%capture
!pip install ollama
!pip install langchain chromadb gradio
!pip install -U langchain-community
!pip install pymupdf

In [2]:
%%capture
# Установка Ollama в Google Colab
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama-python pyngrok

In [3]:
import ollama  # Для взаимодействия с локальными LLM через Ollama
import gradio as gr  # Для создания веб-интерфейса
import os      # Для работы с файловой системой (проверка файлов, путей)
import re      # Для работы с регулярными выражениями (очистка ответа модели)

# Компоненты LangChain для обработки документов и RAG
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, DirectoryLoader # Загрузчики для PDF, TXT и директорий
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Для разбиения текста на чанки
from langchain_community.vectorstores import Chroma  # Векторная база данных ChromaDB
from langchain_community.embeddings import OllamaEmbeddings  # Модель для создания эмбеддингов через Ollama

*   **Пояснение:**
    *   Мы устанавливаем: `ollama` (для запуска LLM локально), `langchain` (фреймворк для LLM-приложений), `chromadb` (векторная база данных), `gradio` (для веб-интерфейса), `pymupdf` (для чтения PDF) и `langchain-community` (дополнительные компоненты LangChain).
    *   Затем мы импортируем необходимые классы и функции:
        *   `ollama`: Для прямого взаимодействия с LLM.
        *   `gradio`: Для создания пользовательского интерфейса.
        *   `re`: Для работы с регулярными выражениями (очистка текста).
        *   `PyMuPDFLoader`: Загрузчик текста из PDF (из `langchain_community`).
        *   `RecursiveCharacterTextSplitter`: Инструмент для разбиения текста на части (из `langchain`).
        *   `Chroma`: Векторная база данных (из `langchain`).
        *   `OllamaEmbeddings`: Модель для создания векторных представлений текста (из `langchain_community`).

### **2. Базовое Взаимодействие с LLM (без RAG)**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В основе современных достижений в обработке естественного языка лежат **большие языковые модели (LLM)**. Как правило, это глубокие нейронные сети, построенные на архитектуре **Трансформер (Transformer)**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Их обучение происходит на гигантских текстовых корпусах с использованием задач самообучения (self-supervised learning), например, предсказания следующего слова. В результате модель накапливает обширные **параметрические знания** о языке, фактах и закономерностях мира, которые кодируются в ее весах.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Способность LLM решать разнообразные задачи часто рассматривается как **эмерджентное свойство**, проявляющееся при достижении моделью определенного масштаба.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Прежде чем строить RAG-систему, посмотрим, как можно напрямую взаимодействовать с локально развернутой LLM с помощью `Ollama`.

In [4]:
# Запуск Ollama в фоновом режиме
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [6]:
# Загрузка моделей в Ollama
%%capture
!ollama pull deepseek-r1:1.5b
!ollama pull deepseek-r1:8b
!ollama pull llama3.1

#### **Тестируем модель "deepseek-r1:1.5b":**

In [7]:
MODEL = "deepseek-r1:1.5b"

In [8]:
# Прямой вызов LLM для ответа на вопрос
llm_response = ollama.chat(
    model=MODEL, # Указываем используемую модель
    messages=[
        {"role": "user", "content": "Объясни второй закон Ньютона на профессиональном русском языке."}, # Запрос пользователя
    ],
)

# Вывод ответа модели
print(llm_response["message"]["content"])

<think>
Хм, пользователь просит объяснить Second Law of Newton на профессиональном русском языке. Я помню, что это законы Письма, которые важны в физике и индустрии. Давайте посмотрим.

Сначала мне нужно понять, о чем Second Law. Оно связано с изменением Momentum. Мне кажется, это means change in momentum is equal to force applied. Но как именно он выглядит на практике?

Поскольку пользователь использует русский язык, я должен его использовать в правильном контексте. Возможно, стоит упомянуть различные темы применения Second Law: для споделывания движения тела и центральной тяжелистости.

Но как лучше выразить это? Может быть, подчеркнуть, что Second Law описывает relationship between force and acceleration, а также, как это может изменять.velocity и позицию.

Также стоит упомянуть, что Second Law не утверждает因果ية, только прямые связи между magnitudes. Это важно для понимания, где можно применить его.

Проверю, есть ли дополнительные моменты, которые могут быть важными: например, связ

#### **Тестируем модель "deepseek-r1:8b":**

In [9]:
MODEL = "deepseek-r1:8b"

In [10]:
# Прямой вызов LLM для ответа на вопрос
llm_response = ollama.chat(
    model=MODEL, # Указываем используемую модель
    messages=[
        {"role": "user", "content": "Объясни второй закон Ньютона на профессиональном русском языке."}, # Запрос пользователя
    ],
)

# Вывод ответа модели
print(llm_response["message"]["content"])

<think>
Хорошо, я пытаюсь разобраться с вторым законом Ньютона. Я помню, что законы Ньютона связаны с运动ом и гравитацией, но я не уверен, как именно они работают. Второй закон, возможно, связан с определением момента импульса или изменения движения тела.

Первое, что приходит в голову, это то, что первый закон Ньютона о том, что силовые矢量 количества массы размягчености равны impulse of change of momentum. То есть F = dp/dt. Это я понял, но как第二ый закон связан с этим?

Пользователь спрашивает на русском языке, так что мне нужно убедиться, что я правильно понимаю термины и объяснения.

Далее, я пытаюсь найти определение второго закона Ньютона. Возможно, он имеет два аспекта: первый касается того, как力 создает изменение момента импульса, а второй — о сохранении количества движения в изолированной системе.

Но я не уверен, правильно ли я это понимаю. Может быть, мне нужно рассмотреть примеры или иллюстрации, чтобы лучше понять.

Если бы я хотел объяснить это на профессиональном русском язы

#### **Тестируем модель "llama3.1":**

In [11]:
MODEL = "llama3.1"

In [12]:
# Прямой вызов LLM для ответа на вопрос
llm_response = ollama.chat(
    model=MODEL, # Указываем используемую модель
    messages=[
        {"role": "user", "content": "Объясни второй закон Ньютона на профессиональном русском языке."}, # Запрос пользователя
    ],
)

# Вывод ответа модели
print(llm_response["message"]["content"])

Второй закон Ньютона гласит, что при действии внешней силы на предмет у него появляется и ускорение пропорциональное величине силы и обратно пропорциональное его массе.

Формально он можно выразить следующим образом:

F = мa,

где F — сила, а m — масса объекта. Аскорбиновое число (a) — это разность между ускорением предмета при наложении силы и его ускорением в отсутствии сил.

Также второй закон Ньютона можно выразить как:

F = м(Δv/Δt),

где Δv — изменение скорости объекта, а Δt — время изменения скорости. 

Этот закон имеет важное значение для понимания движения предметов и их взаимодействия с окружающей средой.

Природа закона заключается в том, что сила, действующая на систему, влияет на ее ускорение, и величина этого ускорения зависит от массы системы.


*   **Пояснение:**
    *   **Большая Языковая Модель (LLM - Large Language Model):** Это нейронная сеть, обученная на огромных объемах текста, способная понимать и генерировать человекоподобный ответ.
    *   Функция `ollama.chat()` отправляет запрос (`messages`) к указанной модели (`model`).
    *   Ответ модели содержится в словаре `llm_response` по ключу `["message"]["content"]`.
    *   На данном этапе ответ генерируется исключительно на основе внутренних, параметрических знаний модели, без учета каких-либо внешних документов.

### **3. Предварительная Обработка PDF-документа для RAG**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Чтобы LLM могла использовать информацию из внешних источников, таких как наши PDF-документы, в RAG-системах применяется механизм **информационного поиска (Information Retrieval, IR)**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Современный подход к поиску в текстовых данных основан на **векторных представлениях (embeddings)**. Текст (слова, предложения, абзацы) преобразуется в числовые векторы в многомерном **латентном пространстве** таким образом, что семантически близкие фрагменты текста оказываются рядом в этом пространстве (например, по косинусному расстоянию).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Для хранения таких векторов и быстрого поиска по сходству используются **векторные базы данных**. Они применяют эффективные алгоритмы **приближенного поиска ближайших соседей (Approximate Nearest Neighbors, ANN)**, например, на основе графовых (HNSW) или кластерных (IVF) индексов. Перед созданием векторов исходный документ обычно разбивают на **чанки (chunks)** – небольшие фрагменты.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Это необходимо, во-первых, из-за ограничения на длину входного текста, который LLM может обработать за раз (**контекстное окно**), а во-вторых, для повышения точности поиска, позволяя находить более гранулярные и релевантные части документа.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Подготовим функцию, которая выполнит все эти шаги для нашего PDF.

In [21]:
# Функция обработки PDF: загрузка, разбиение, эмбеддинг, сохранение в ChromaDB
def process_pdf_for_rag(pdf_filepath):
    """Загружает PDF, разбивает на чанки, создает эмбеддинги и ретривер."""
    if pdf_filepath is None:
        print("PDF-файл не предоставлен.")
        return None, None # Возвращаем None для vector_database и document_retriever

    # 1. Загрузка текста из PDF
    loader = PyMuPDFLoader(pdf_filepath)
    loaded_documents = loader.load()
    if not loaded_documents:
        print("Не удалось извлечь текст из PDF.")
        return None, None

    # 2. Разбиение текста на чанки
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    document_chunks = text_splitter.split_documents(loaded_documents)

    # 3. Создание модели эмбеддингов
    embedding_model = OllamaEmbeddings(model=MODEL)

    # 4. Создание и наполнение векторной базы данных
    # persist_directory - папка для сохранения БД на диске
    vector_database = Chroma.from_documents(
        documents=document_chunks,
        embedding=embedding_model,
        persist_directory="chroma_db_rag_concise/"
    )

    # 5. Создание ретривера
    document_retriever = vector_database.as_retriever()

    print(f"PDF обработан: {len(document_chunks)} чанков сохранено в ChromaDB.")
    return vector_database, document_retriever # Возвращаем только то, что нужно дальше

*   **Пояснение:**
    *   Функция `process_pdf_for_rag` принимает путь к PDF-файлу (`pdf_filepath`).
    *   **Шаг 1: Загрузка:** `PyMuPDFLoader` извлекает текст из PDF.
    *   **Шаг 2: Разбиение:** `RecursiveCharacterTextSplitter` делит текст на **чанки (chunks)** - небольшие фрагменты (здесь по 500 символов с перекрытием 100 символов). Это необходимо из-за ограничения на объем текста, который LLM может обработать за раз (контекстное окно), а перекрытие помогает сохранить связи между чанками.
    *   **Шаг 3: Эмбеддинги:** `OllamaEmbeddings` инициализирует модель для преобразования текста в **векторные представления (embeddings)**. Эмбеддинги - это числовые векторы, отражающие семантическое значение текста. Похожие по смыслу тексты имеют близкие векторы.
    *   **Шаг 4: Векторная БД:** `Chroma.from_documents` создает **векторную базу данных (`vector_database`)**. Она сохраняет чанки и их эмбеддинги, позволяя эффективно искать похожие по смыслу фрагменты. `persist_directory` указывает, где хранить базу данных локально.
    *   **Шаг 5: Ретривер:** `vector_database.as_retriever()` создает **ретривер (`document_retriever`)**. Это объект, который будет выполнять поиск наиболее релевантных чанков в базе данных по запросу пользователя.
    *   Функция возвращает созданную базу данных и ретривер.

### **4. Объединение найденных фрагментов документа**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;На этапе поиска (retrieval) система извлекает из базы знаний набор фрагментов (чанков), наиболее релевантных запросу пользователя.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Поскольку LLM обрабатывает входные данные как единую последовательность, эти фрагменты нужно объединить в связный текст. Этот объединенный контекст затем добавляется к исходному запросу.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Качество и структура этого объединенного текста напрямую влияют на способность модели эффективно использовать предоставленную информацию.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Важно также следить, чтобы итоговый текст не превышал максимальную длину **контекстного окна (context window)** модели. Создадим простую функцию для такого объединения.

In [22]:
# Функция для объединения текстов из списка документов LangChain
def combine_retrieved_documents(retrieved_documents):
    """Объединяет page_content найденных документов в одну строку."""
    return "\n\n".join(doc.page_content for doc in retrieved_documents)

*   **Пояснение:**
    *   Ретривер возвращает список документов (чанков). Эта функция извлекает текстовое содержимое (`page_content`) каждого документа и соединяет их через двойной перенос строки (`\n\n`), формируя единый контекст для LLM.

### **5. Запрос к LLM с Дополненным Контекстом**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Правильное формирование входных данных для LLM, известное как **инженерия промптов (prompt engineering)**, играет ключевую роль в получении качественных результатов. В RAG-системах промпт обычно строится так, чтобы четко разделить исходный вопрос пользователя и извлеченный из документов контекст. Это помогает модели понять, что ответ следует генерировать, опираясь на предоставленную информацию.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Такой механизм можно рассматривать как форму **обучения в контексте (in-context learning)**, когда модель адаптирует свое поведение на основе инструкций или примеров, данных во входном промпте, без изменения своих весов. Явное указание ролей ("Вопрос:", "Контекст:") направляет модель на решение конкретной задачи.

In [23]:
# Функция для запроса к LLM с использованием контекста
def query_llm_with_context(user_question, retrieved_context):
    """Формирует промпт и вызывает LLM, передавая вопрос и контекст."""

    # Формирование промпта для LLM
    llm_prompt = f"Question: {user_question}\n\nContext: {retrieved_context}\n\nAnswer:"

    # Вызов LLM
    response = ollama.chat(
        model=MODEL,
        messages=[{'role': 'user', 'content': llm_prompt}]
    )

    # Извлечение и очистка ответа
    raw_response_content = response['message']['content']
    # Удаление тегов <think>...</think>, если они есть
    cleaned_response = re.sub(r'<think>.*?</think>', '', raw_response_content, flags=re.DOTALL).strip()

    return cleaned_response

*   **Пояснение:**
    *   **Промпт (Prompt):** Это структурированная инструкция или запрос, подаваемый LLM. Здесь `llm_prompt` содержит и вопрос пользователя (`user_question`), и найденный контекст (`retrieved_context`), чтобы модель генерировала ответ на основе предоставленной информации.
    *   Функция отправляет промпт модели `deepseek-r1:1.5b`.
    *   Ответ извлекается и очищается от возможных вспомогательных тегов (например, `<think>...</think>`, которые модель использует для "рассуждений") с помощью регулярного выражения.

### **6. Построение RAG-пайплайна**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Подход **Retrieval-Augmented Generation (RAG)** сочетает **параметрические знания**, хранящиеся в весах LLM, с **непараметрическими знаниями**, которые динамически извлекаются из внешнего источника данных во время работы модели (**инференса**).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В отличие от дообучения (fine-tuning), RAG позволяет использовать актуальную или специфическую информацию без необходимости дорогостоящего переобучения LLM.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Типичный RAG-пайплайн включает шаги: получение запроса, поиск релевантных документов в базе знаний, дополнение запроса найденной информацией и генерация ответа LLM на основе этого дополненного запроса.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Фреймворки, такие как `LangChain`, предлагают удобные средства для построения и запуска таких многошаговых процессов. Соберем все наши компоненты в единую RAG-цепочку.

In [24]:
# Функция, реализующая RAG-пайплайн
def rag_chain(user_question, document_retriever):
    """Выполняет поиск релевантных документов и генерирует ответ LLM с их учетом."""

    # 1. Поиск релевантных документов (Retrieval)
    relevant_documents = document_retriever.invoke(user_question)

    # 2. Форматирование контекста
    combined_context = combine_retrieved_documents(relevant_documents)

    # 3. Генерация ответа с контекстом (Generation)
    final_answer = query_llm_with_context(user_question, combined_context)

    return final_answer

In [25]:
# Функция-обертка для Gradio интерфейса
def answer_question_from_pdf(pdf_file_object, user_question):
    """Обрабатывает PDF (если есть) и отвечает на вопрос с помощью RAG."""
    vector_database = None
    document_retriever = None

    if pdf_file_object is not None:
        pdf_filepath = pdf_file_object.name # Получаем путь к временному файлу Gradio
        print(f"Обработка файла: {pdf_filepath}")
        try:
            # Обрабатываем PDF и получаем ретривер
            vector_database, document_retriever = process_pdf_for_rag(pdf_filepath)
            if document_retriever is None:
                 return "Ошибка: Не удалось обработать PDF."
        except Exception as e:
            print(f"Ошибка при обработке PDF: {e}")
            return f"Ошибка при обработке PDF: {e}. Убедитесь, что файл корректен."
    else:
        # Если PDF не загружен, RAG невозможен для этого запроса
         return "Пожалуйста, загрузите PDF-файл, чтобы задать вопрос по его содержимому."

    if not user_question:
        return "Пожалуйста, введите вопрос."

    # Если PDF обработан успешно, выполняем RAG
    print(f"Выполнение RAG для вопроса: '{user_question}'")
    try:
        result = rag_chain(user_question, document_retriever)
        return result
    except Exception as e:
        print(f"Ошибка при выполнении RAG: {e}")
        return f"Ошибка при генерации ответа: {e}"


*   **Пояснение:**
    *   **RAG-пайплайн (RAG Pipeline/Chain):** Это последовательность действий: поиск релевантной информации -> дополнение запроса этой информацией -> генерация ответа LLM.
    *   Функция `rag_chain` реализует этот пайплайн:
        1.  `document_retriever.invoke(user_question)`: Использует ретривер для поиска в векторной базе данных чанков, наиболее релевантных вопросу пользователя.
        2.  `combine_retrieved_documents`: Объединяет найденные чанки в единый контекст.
        3.  `query_llm_with_context`: Вызывает LLM с вопросом и сформированным контекстом.
    *   Функция `answer_question_from_pdf` является основной логикой для Gradio:
        *   Она принимает объект файла (`pdf_file_object`) и вопрос (`user_question`).
        *   Если файл загружен, она извлекает путь к нему (`pdf_file_object.name`) и вызывает `process_pdf_for_rag` для создания/загрузки векторной базы и ретривера. Добавлена обработка ошибок на случай проблем с PDF.
        *   Если файл не загружен или произошла ошибка при его обработке, RAG не выполняется.
        *   Если ретривер успешно создан и вопрос задан, вызывается `rag_chain` для получения ответа. Добавлена обработка ошибок на случай проблем при выполнении RAG.
        *   Возвращает сгенерированный ответ или сообщение об ошибке.

### **7. Создание Интерфейса Чат-бота с Помощью Gradio**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Важной частью разработки прикладных систем ИИ является создание удобных средств взаимодействия с пользователем (**человеко-машинное взаимодействие, Human-Computer Interaction, HCI**).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Инструменты вроде `Gradio` позволяют быстро создавать интерактивные веб-интерфейсы для демонстрации работы моделей машинного обучения, проведения экспериментов и сбора обратной связи. Предоставление простого интерфейса для таких систем, как наш RAG-пайплайн, делает технологию доступнее и упрощает ее тестирование. Создадим такой интерфейс.

In [26]:
# Создание и запуск интерфейса Gradio
chatbot_interface = gr.Interface(
    fn=answer_question_from_pdf, # Функция, обрабатывающая ввод/вывод
    inputs=[
        gr.File(label="Загрузите PDF", file_types=['.pdf']), # Поле для загрузки PDF
        gr.Textbox(label="Задайте вопрос по документу")    # Поле для ввода вопроса
    ],
    outputs=gr.Textbox(label="Ответ"), # Поле для вывода ответа
    title=f"Чат-бот для PDF (RAG + {MODEL})",
    description="Загрузите PDF и задайте вопрос. Модель ответит, используя содержимое документа.",
    allow_flagging='never'
)

# Запуск интерфейса
print("Запуск интерфейса Gradio...")
chatbot_interface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Запуск интерфейса Gradio...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c09e7bb15ddf5a4da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


*   **Пояснение:**
    *   `gr.Interface` создает веб-приложение.
    *   `fn`: Основная функция (`answer_question_from_pdf`), которая будет вызываться.
    *   `inputs`: Список компонентов для ввода данных (загрузка файла `gr.File` и текстовое поле `gr.Textbox`). `file_types=['.pdf']` ограничивает тип загружаемых файлов.
    *   `outputs`: Компонент для вывода результата (`gr.Textbox`).
    *   `title`, `description`: Заголовок и описание интерфейса.
    *   `chatbot_interface.launch()`: Запускает локальный веб-сервер, делая интерфейс доступным в браузере.

### **8. Удаление Установленных Компонентов (Опционально)**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В процессе разработки важно уметь управлять установленными программными компонентами и созданными артефактами (моделями, базами данных). Корректное удаление неиспользуемых элементов освобождает системные ресурсы и помогает поддерживать порядок в рабочей среде. Ниже приведены инструкции по удалению основных компонентов, которые мы использовали.

*   **Удаление Ollama:**
    *   macOS: Удалите приложение Ollama из папки "Программы".
    *   Windows: Используйте "Установка и удаление программ".
    *   Linux: Следуйте инструкциям для вашего метода установки.
*   **Удаление моделей Ollama (например, DeepSeek):**
    *   Найдите и удалите папку `.ollama/models` в вашей домашней директории (`~/.ollama/models` на macOS/Linux, `C:\Users\%username%\.ollama\models` на Windows).
*   **Удаление векторной базы данных:**
    *   Удалите папку, указанную в `persist_directory` (в нашем примере `./chroma_db_rag_concise`).
*   **Удаление Python-пакетов:**
    *   `pip uninstall ollama langchain chromadb gradio pymupdf langchain-community`

---